In [37]:
from scipy.sparse import csr_matrix
I = [1, 1, 2, 2, 3, 3, 4, 6, 7, 8, 9, 3, 5, 6, 7, 3, 5, 7, 4, 5, 3, 4]
J = [5, 6, 7, 3, 6, 2, 9, 5, 3, 5, 6, 1, 1, 2, 2, 3, 3, 4, 6, 7, 8, 9]
V = [5]*len(I)
A = csr_matrix((V, (I, J)), shape=(10, 10))
A.todense(), A.shape

(matrix([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  5,  5,  0,  0,  0],
         [ 0,  0,  0,  5,  0,  0,  0,  5,  0,  0],
         [ 0,  5,  5,  5,  0,  0,  5,  0,  5,  0],
         [ 0,  0,  0,  0,  0,  0,  5,  0,  0, 10],
         [ 0,  5,  0,  5,  0,  0,  0,  5,  0,  0],
         [ 0,  0,  5,  0,  0,  5,  0,  0,  0,  0],
         [ 0,  0,  5,  5,  5,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  5,  0,  0,  0]], dtype=int64), (10, 10))

In [61]:
B = np.zeros((A.shape[0],A.shape[1],5), dtype=int)
A.shape, B.shape

((10, 10), (10, 10, 5))

In [31]:
B = A.T
# S_g = A
# S_gT = B
# diff = 10
# csr_matrix((S_gT.data, S_gT.indices, S_gT.indptr), shape=(S_gT.shape[0], S_gT.shape[1] + diff)).T.todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [9]:
B = A[[2, 3, 4], :][:, [2, 3, 4]]
B.todense()

matrix([[0, 1, 0],
        [1, 0, 0],
        [0, 0, 0]], dtype=int64)

In [79]:
np.array(A.sum(axis=0)).ravel().shape

(10,)

In [10]:
B[4, 4]

IndexError: index out of bounds: 0<=4<3, 0<=4<3

In [62]:
import torch
x = torch.from_numpy(A.todense())
y = torch.from_numpy(B)

In [39]:
class MySpMM(torch.autograd.Function):

    @staticmethod
    def forward(ctx, sp_mat, dense_mat):
        ctx.save_for_backward(sp_mat, dense_mat)

        return torch.mm(sp_mat, dense_mat)

    @staticmethod
    def backward(ctx, grad_output):        
        sp_mat, dense_mat = ctx.saved_variables
        grad_matrix1 = grad_matrix2 = None

        assert not ctx.needs_input_grad[0]
        if ctx.needs_input_grad[1]:
            grad_matrix2 = Variable(torch.mm(sp_mat.data.t(), grad_output.data))
        
        return grad_matrix1, grad_matrix2

def gnn_spmm(sp_mat, dense_mat):
    return MySpMM.apply(sp_mat, dense_mat)

In [63]:
gnn_spmm(x, y)

RuntimeError: matrices expected, got 2D, 3D tensors at /pytorch/aten/src/TH/generic/THTensorMath.cpp:192

In [91]:
import networkx as nx
S = A[range(5), :]
B = nx.bipartite.from_biadjacency_matrix(S)
nx.get_node_attributes(B, 'bipartite')

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1}

In [130]:
np.ones(5)*np.inf

array([inf, inf, inf, inf, inf])

In [119]:
u, v = 2, 3

In [131]:
B = S.T * S

In [132]:
B

<10x10 sparse matrix of type '<class 'numpy.int64'>'
	with 34 stored elements in Compressed Sparse Column format>